In [1]:
# Library header
from PIL import Image
from matplotlib import pyplot as plt
from pathlib import Path
from tensorflow.keras import Model, Input
from tqdm import tqdm
import shutil #library for moving files
import tensorflow as tf
import numpy as np
import cv2

# own libraries
import split_dataset as spData
import handseg_two_classes as hand
import data_gen as dg


#globals
input_width = 640
input_height = 480
handseg_path = '../../../handseg-150k'

0.2.9


In [ ]:
# preprocessing
def preprocess(img):
    return (img / img.max()) * 255

# create the data generators with data_loader
# ... for training
train_gen = dg.image_segmentation_generator(
        images_path=handseg_path+"/images/",
        segs_path=handseg_path+"/masks/", 
        batch_size=32,
        n_classes=2,
        input_height=480,
        input_width=640,
        output_height=480,
        output_width=640,
        do_augment=False,
        preprocessing=preprocess,
        read_image_type=1)
# read_image_type -> 0 = grayscale; -> 1 = rgb

# ... for validation
val_gen = dg.image_segmentation_generator(
        images_path=handseg_path+"/val_images/",
        segs_path=handseg_path+"/val_masks/",
        batch_size=32,
        n_classes=2,
        input_height=480,
        input_width=640,
        output_height=480,
        output_width=640,
        preprocessing=preprocess,
        read_image_type=1)

# ... for testing
test_gen = dg.image_segmentation_generator(
        images_path=handseg_path+"/test_images/",
        segs_path=handseg_path+"/test_masks/",
        batch_size=2,
        n_classes=2,
        input_height=480,
        input_width=640,
        output_height=480,
        output_width=640,
        preprocessing=preprocess,
        read_image_type=1)

In [ ]:
# input is a (480, 640, 2) np.array -> interpret the softmax values in third dimension
def interpretPrediction(pred):
  # create np.array to save the interpretation
  interpretation = np.zeros((480, 640))
  height, width, _ = pred.shape
  for i in range(height):
    for j in range(width):
      # create a list of the probabilites from each class
      value_list = [pred[i,j,0], pred[i,j,1]]
      # find out, which class has been predicted
      largest = max(value_list)
      index_of_largest = value_list.index(largest)
      # store new value in the interpretation array
      interpretation[i,j] = index_of_largest
  return interpretation

def drawBoxes(mask, img):
  # Iterate all colors in mask
  for color in np.unique(mask):

      # Color 0 is assumed to be background or artifacts
      if color == 0:
          continue

      # Determine bounding rectangle w.r.t. all pixels of the mask with
      # the current color
      x, y, w, h = cv2.boundingRect(np.uint8(mask == color))
      print(x,y,w,h)
      # Draw bounding rectangle to color image
      out = cv2.rectangle(img.copy(), (x, y), (x+w, y+h), (255, int(color), 0), 2)

      # Show image with bounding box
      plt.imshow(out, cmap="Greys"); plt.title('img_' + str(color)); plt.show()

In [ ]:
# load the weights from previous training
handseg.load_weights("./handseg_model_twoClasses_3.h5")

# training
history = handseg.fit(x=train_gen,
            epochs=10,
            batch_size=32,
            steps_per_epoch=512,
            validation_data=val_gen,
            validation_batch_size=32,
            validation_steps=256)

# load/save the already trained weigths
handseg.save_weights("./handseg_model_twoClasses_updated.h5")

In [ ]:
# evaluation
# returns a list of tupels containing the values for iou and accuracy
# since the metrics are updated in each step, the las tupel value containst the
# final result
# load the weights from previous training
handseg.load_weights("./handseg_model_twoClasses_1.h5")

def eval_handseg():
  # path values for test data
  paths_test_images = glob.glob(handseg_path+"/test_images/*.png")
  paths_test_mask = glob.glob(handseg_path+"/test_masks/*.png")

  # create the respective metrics
  metric_iou = tf.keras.metrics.MeanIoU(num_classes=2)
  metric_acc = tf.keras.metrics.Accuracy()

  # initialize list
  result = []

  for i in tqdm(range(len(paths_test_images))):
    # load the img
    in_image = cv2.imread(paths_test_images[i])
    in_image = np.reshape(in_image, (1,480, 640, 3))
    # apply preprocessing
    in_image = preprocess(in_image)
    # predict
    out_im = handseg.predict(in_image)
    # interpret prediction
    out_im = interpretPrediction(out_im[0])
    # calculate IoU and accuracy
    gt = cv2.imread(paths_test_mask[i])
    metric_iou.update_state(gt[:,:,0], out_im)
    metric_acc.update_state(gt[:,:,0], out_im)
    # append tupel to list
    result.append((metric_iou.result().numpy(), metric_acc.result().numpy()))

  return result

res = eval_handseg()
# print results -> last element in list show the result
print(res[-1])